Import necessary libraries

In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator


Download and extract the Caltech-256 dataset

In [2]:
!wget https://caltech256-bucket.s3.amazonaws.com/256_ObjectCategories.tar
!tar -xf 256_ObjectCategories.tar



--2023-05-26 03:29:20--  https://caltech256-bucket.s3.amazonaws.com/256_ObjectCategories.tar
Resolving caltech256-bucket.s3.amazonaws.com (caltech256-bucket.s3.amazonaws.com)... 52.217.132.249, 52.217.133.121, 52.216.113.59, ...
Connecting to caltech256-bucket.s3.amazonaws.com (caltech256-bucket.s3.amazonaws.com)|52.217.132.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1183006720 (1.1G) [application/x-tar]
Saving to: ‘256_ObjectCategories.tar.2’

256_ObjectCategorie 100%[===================>]   1.10G  55.3MB/s    in 18s     

2023-05-26 03:29:39 (61.6 MB/s) - ‘256_ObjectCategories.tar.2’ saved [1183006720/1183006720]



Define dataset path and load the data

In [3]:
import glob
from sklearn.preprocessing import LabelEncoder

dataset_path = './256_ObjectCategories'
all_images = glob.glob(f'{dataset_path}/*/*.jpg')
all_labels = [os.path.basename(os.path.dirname(img)) for img in all_images]

# Train-test split
train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=42, stratify=all_labels)


Create custom data generators

In [4]:
img_size = 224
batch_size = 32

def train_data_generator(images, labels, batch_size):
    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels)
    num_classes = len(encoder.classes_)
    
    while True:
        idx = np.random.permutation(len(images))
        for i in range(0, len(images), batch_size):
            batch_idx = idx[i:i+batch_size]
            batch_images = [tf.keras.preprocessing.image.load_img(img, target_size=(img_size, img_size)) for img in np.array(images)[batch_idx]]
            
            batch_x = np.array([tf.keras.preprocessing.image.img_to_array(img,data_format='channels_last') for img in batch_images]) 

            batch_y = tf.keras.utils.to_categorical(encoded_labels[batch_idx], num_classes)

            yield batch_x, batch_y

def test_data_generator(images, labels, batch_size):
    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels)
    num_classes = len(encoder.classes_)
    
    while True:
        idx = np.arange(len(images))
        for i in range(0, len(images), batch_size):
            batch_idx = idx[i:i+batch_size]
            batch_images = [tf.keras.preprocessing.image.load_img(img, target_size=(img_size, img_size)) for img in np.array(images)[batch_idx]]
            
            batch_x = np.array([tf.keras.preprocessing.image.img_to_array(img,data_format='channels_last') for img in batch_images]) 
            batch_y = tf.keras.utils.to_categorical(encoded_labels[batch_idx], num_classes)
            yield batch_x, batch_y

train_generator = train_data_generator(train_images, train_labels, batch_size)
test_generator = test_data_generator(test_images, test_labels, batch_size)

Define a simple CNN model

In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization

base_model = ResNet50(include_top=False, pooling = 'avg', input_shape = (224, 224 ,3), weights = 'imagenet')
base_model.trainable = False

inputs = Input(shape=(224,224,3))
x = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)(inputs)
x = tf.keras.applications.resnet50.preprocess_input(inputs)
x = base_model(x, training = False)
x = Flatten()(x)
outputs = Dense(257, activation = 'softmax')(x)
modeel_res = tf.keras.Model(inputs,outputs)

In [6]:
modeel_res.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 257)               526593    
                                                             

In [7]:
from tensorflow.keras.optimizers.legacy import Adam
modeel_res.compile(optimizer=Adam(learning_rate = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Train the CNN model

In [8]:
epochs = 10
steps_per_epoch = len(train_images) // batch_size
validation_steps = len(test_images) // batch_size

history = modeel_res.fit(train_generator, epochs=epochs,  steps_per_epoch = steps_per_epoch, validation_data=test_generator, validation_steps=validation_steps, verbose=1)


Epoch 1/10
765/765 [==============================] - 178s 223ms/step - loss: 1.4219 - accuracy: 0.6879 - val_loss: 0.8907 - val_accuracy: 0.7868
Epoch 2/10
765/765 [==============================] - 170s 222ms/step - loss: 0.3709 - accuracy: 0.8978 - val_loss: 0.8395 - val_accuracy: 0.7976
Epoch 3/10
765/765 [==============================] - 170s 222ms/step - loss: 0.1619 - accuracy: 0.9560 - val_loss: 0.8295 - val_accuracy: 0.8099
Epoch 4/10
765/765 [==============================] - 161s 211ms/step - loss: 0.0742 - accuracy: 0.9832 - val_loss: 0.8117 - val_accuracy: 0.8172
Epoch 5/10
765/765 [==============================] - 170s 223ms/step - loss: 0.0423 - accuracy: 0.9917 - val_loss: 0.8172 - val_accuracy: 0.8210
Epoch 6/10
765/765 [==============================] - 171s 224ms/step - loss: 0.0256 - accuracy: 0.9959 - val_loss: 0.8608 - val_accuracy: 0.8182
Epoch 7/10
765/765 [==============================] - 158s 206ms/step - loss: 0.0424 - accuracy: 0.9897 - val_loss: 1.0402 -

Evaluate the model on the test set

In [10]:
test_loss, test_accuracy = modeel_res.evaluate(test_generator, steps=validation_steps)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")


191/191 [==============================] - 31s 162ms/step - loss: 1.1219 - accuracy: 0.8107
Test loss: 1.1219
Test accuracy: 0.8107
